In [2]:
from collections import Counter
from flask import Flask, request, render_template, jsonify
import pickle
import pandas as pd
import numpy as np
import requests
import re
from bs4 import BeautifulSoup
import argparse
import time
from splinter import Browser
import time
from collections import defaultdict
import json
import re

In [12]:
f = open('.env','r')
env = {}
for line in f.readlines():
    env[line.strip().split("=")[0]]=line.strip().split("=")[1]

In [255]:
br = Browser("firefox")
br.visit("https://www.fantasycruncher.com/login?referer=/")
br.find_by_id('user_email').fill(env['fc_login'])
br.find_by_id('user_password').fill(env['fc_pw'])
br.find_by_id('submit').click()

In [256]:
rewind_base = "https://www.fantasycruncher.com/lineup-rewind/draftkings/NBA/"
date = "2018-10-29"
br.visit(rewind_base+date)

In [257]:
all_players = "/html/body/div[3]/div[1]/div[1]/div/div[2]/div[8]/div[2]/div[2]/div[2]/div[2]/div/label/select/option[7]"
br.find_by_name("ff_length").click()
br.find_by_xpath(all_players).click()

In [243]:
nav_links_xpath = "/html/body/div[2]/header/ul/li"
nav = br.find_by_xpath(nav_links_xpath)

In [40]:
sport = "NBA"
for ind, elem in enumerate(nav):
    print(elem.find_by_css('.top-nav-link').value)
    if (elem.find_by_css('.top-nav-link').value == sport):
        print("click")
        br.find_by_xpath(nav_links_xpath+"[{}]".format(ind+1)).click()


NFL
NBA
click
MLB
NHL
CFB
GOLF
MMA
CFL
NAS
WNBA
SOCbeta



In [93]:
tr_selector = "/html/body/div[3]/div[1]/div[1]/div/div[2]/div[8]/div[1]/div[2]/table/tbody/tr"
table = br.find_by_xpath(tr_selector)

In [163]:
asdf = table[2]
len(table)

40

In [104]:
asdf.find_by_css(".player-stats").text

'Karl-Anthony Towns'

In [105]:
asdf.find_by_css(".salaryCol").text

'8000'

In [171]:
for date in dates[0:2]:
    print(rewind_base+date)

https://www.fantasycruncher.com/lineup-rewind/draftkings/NBA/2017-10-17
https://www.fantasycruncher.com/lineup-rewind/draftkings/NBA/2017-10-18


In [219]:
dates = data[data['date']>'2017-09-01']['date'].unique()
salary_data = pd.DataFrame()
for date in dates:
    time.sleep(1)
    br.visit(rewind_base+date)
    table = br.find_by_xpath(tr_selector)
    try:
        if len(table) > 0:
            temp_dict = {}
            temp_dict['player'] = []
            temp_dict['pos'] = []
            temp_dict['team']=[]
            temp_dict['opp']=[]
            temp_dict['salary'] = []
            temp_dict['start'] = []
            temp_dict['mins'] = []
            temp_dict['score'] = []
            for ind, row in enumerate(table):
                temp_dict['player'].append(row.find_by_css(".player-stats").text)
                temp_dict['salary'].append(row.find_by_css(".salaryCol").text)
                try:
                    temp_dict['pos'].append(row.find_by_css("td select.pos-select option")[0].text)
                except:
                    temp_dict['pos'].append(row.find_by_css("td")[1].text)
                temp_dict['team'].append(row.find_by_css("td")[2].text)
                temp_dict['opp'].append(row.find_by_css("td")[3].text)
                temp_dict['start'].append(row.find_by_css("td")[-4].text)
                temp_dict['mins'].append(row.find_by_css("td")[-3].text)
                temp_dict['score'].append(row.find_by_css("td")[-2].text)                              
            temp_df = pd.DataFrame(temp_dict)
            temp_df['date']=date
            salary_data = salary_data.append(temp_df)
        else:
            print("empty {}".format(date))
    except:
        print("error {}".format(date))
print("Done!")


error 2017-10-18


MaxRetryError: HTTPConnectionPool(host='127.0.0.1', port=59169): Max retries exceeded with url: /session/45804187-4eec-594c-ac8f-394e2e238f99/url (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x12fd0f2e8>: Failed to establish a new connection: [Errno 61] Connection refused'))

In [287]:
dates = data[data['date']>'2017-09-01']['date'].unique()
date = dates[1]
br.visit(rewind_base+date)
table = br.find_by_xpath(tr_selector)

if len(table) > 0:
    len_table = len(table)
    temp_dict = {}
    temp_dict['player'] = []
    temp_dict['pos'] = []
    temp_dict['salary'] = []
    for ind, row in enumerate(table):
        now = time.time()
        temp_dict['player'].append(row.find_by_css(".player-stats").text)
        temp_dict['salary'].append(row.find_by_css(".salaryCol").text)
        print("{:.02f}".format(time.time()-now))
        try:
            temp_dict['pos'].append(row.find_by_css("td select.pos-select option")[0].text)
        except:
            temp_dict['pos'].append(row.find_by_css("td")[1].text)
        print("{:.02f}".format(time.time()-now))
    temp_df = pd.DataFrame(temp_dict)
    temp_df['date']=date
    salary_data = salary_data.append(temp_df)

0.21
0.25
0.04
0.31
0.04
0.07
0.04
0.31
0.03
0.07
0.04
0.31
0.04
0.31
0.14
0.18
0.05
0.35
0.06
0.36
0.04
0.33
0.04
0.07
0.04
0.32
0.04
0.32
0.04
0.32
0.04
0.32
0.04
0.32
0.04
0.33
0.04
0.07
0.04
0.33
0.04
0.34
0.04
0.34
0.04
0.33
0.04
0.07
0.04
0.32
0.04
0.33
0.04
0.33
0.04
0.33
0.04
0.35
0.04
0.07
0.04
0.34
0.05
0.35
0.04
0.35
0.04
0.34
0.04
0.08
0.04
0.34
0.04
0.08
0.04
0.34
0.04
0.34
0.04
0.34
0.04
0.08
0.04
0.35
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.35
0.04
0.35
0.04
0.08
0.04
0.08
0.04
0.35
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.36
0.04
0.08
0.04
0.35
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.09
0.04
0.08
0.04
0.36
0.04
0.09
0.04
0.36
0.04
0.36
0.04
0.08
0.04
0.37
0.04
0.36
0.04
0.08
0.04
0.37
0.04
0.37
0.05
0.09
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.09
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.09
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.08
0.04
0.37
0.04
0.08
0.04
0.08
0.04
0.08


In [264]:
salary_data.sort_values(by = 'date',inplace = True)

In [286]:
dates = data[data['date']>'2017-09-01']['date'].unique()
date = dates[0]
br.visit(rewind_base+date)
table = br.find_by_xpath(tr_selector)

if len(table) > 0:
    len_table = len(table)
    temp_dict = {}
    temp_dict['player'] = np.array(["blank"]*len_table)
    temp_dict['pos'] = np.array(["blank"]*len_table)
    temp_dict['salary'] = np.array([3000]*len_table)
    for ind, row in enumerate(table):
        now = time.time()
        temp_dict['player'][ind]=row.find_by_css(".player-stats").text
        temp_dict['salary'][ind]=row.find_by_css(".salaryCol").text
        print("{:.02f}".format(time.time()-now))
        #temp_dict['salary'].append(row.find_by_css(".salaryCol").text)
        try:
            temp_dict['pos'][ind]=row.find_by_css("td select.pos-select option")[0].text
        except:
            temp_dict['pos'][ind]=row.find_by_css("td")[1].text
        print("{:.02f}".format(time.time()-now))
    temp_df = pd.DataFrame(temp_dict)
    temp_df['date']=date
    #salary_data = salary_data.append(temp_df)

0.16
0.21
0.03
0.26
0.06
0.10
0.03
0.24
0.04
0.25
0.04
0.23
0.03
0.06
0.03
0.06
0.03
0.24
0.03
0.25
0.03
0.25
0.03
0.23
0.03
0.25
0.03
0.06
0.03
0.24
0.03
0.06
0.03
0.24
0.03
0.06
0.03
0.06
0.03
0.22
0.03
0.05
0.03
0.26
0.04
0.08
0.04
0.07
0.04
0.27
0.03
0.06
0.03
0.06
0.03
0.06
0.03
0.26
0.04
0.28
0.03
0.07
0.04
0.07
0.03
0.06
0.04
0.07
0.04
0.30
0.04
0.07
0.04
0.29
0.03
0.06
0.03
0.06
0.03
0.26
0.03
0.26
0.03
0.27
0.03
0.06
0.03
0.06
0.03
0.06
0.03
0.07
0.03
0.06
0.03
0.27
0.03
0.30
0.04
0.31
0.04
0.07
0.04
0.31
0.04
0.30
0.03
0.27
0.03
0.27
0.03
0.06
0.03
0.27
0.04
0.27
0.03
0.06
0.03
0.06
0.03
0.29
0.03
0.27
0.03
0.29
0.04
0.07
0.04
0.30
0.03
0.29
0.04
0.08
0.04
0.30
0.04
0.07
0.03
0.29
0.03
0.30
0.04
0.07
0.03
0.07
0.03
0.07
0.04
0.30
0.04
0.31
0.03
0.32
0.04
0.30
0.03
0.29
0.03
0.29


In [282]:
asdf = np.array(["blank"]*len_table)
asdf[0]='hey'
asdf[0]

'hey'

In [277]:
row.find_by_css(".player-stats").text

'Darrun Hilliard'

In [280]:
temp_dict['player'][330] = 'asdf'
temp_dict['player'][330]

'a'

In [273]:
asdf[0]

'a'

In [265]:
salary_data.to_csv('')

,player,pos,salary,date
140,Jonathan Isaac,SF/PF,4000,2017-10-18
139,DeMarre Carroll,SF/PF,4000,2017-10-18
138,Cory Joseph,PG,4000,2017-10-18
149,Timofey Mozgov,C,3900,2017-10-18
219,Shabazz Muhammad,SG/SF,3200,2017-10-18


In [260]:
#salary_data = salary_data.append(salary_data2)\
#salary_data = salary_data.iloc[:,['player','pos','salary','date']]
last_end = salary_data['date'].iloc[-1]
last_end
salary_data.tail()

KeyError: 'date'

In [266]:
salary_data.to_csv('data/salary_data.csv',index = False)

In [213]:
pd.read_csv('data/salary_data.csv')

,player,pos,salary,start,mins,score,date
0,LeBron James,SF/PF,10200,Y,41.0,66.50,2017-10-17
1,James Harden,SG,10000,Y,36.0,53.50,2017-10-17
2,Kevin Durant,SF/PF,9600,Y,37.0,41.75,2017-10-17
3,Stephen Curry,PG,8800,Y,29.0,36.75,2017-10-17
4,Kyrie Irving,PG,8300,Y,39.0,50.50,2017-10-17
5,Chris Paul,PG,8000,Y,33.0,36.00,2017-10-17
6,Kevin Love,PF/C,7800,Y,28.0,29.75,2017-10-17
7,Draymond Green,PF/C,7000,Y,27.0,43.25,2017-10-17
8,Gordon Hayward,SF,6700,Y,5.0,3.25,2017-10-17
9,Isaiah Thomas,PG,6500,NaN,0.0,0.00,2017-10-17


In [233]:
last_end = salary_data['date'].iloc[-1]
salary_data = pd.DataFrame()
print(last_end)
dates = data[data['date']>last_end]['date'].unique()
salary_data2 = pd.DataFrame()
for date in dates:
    br.visit(rewind_base+date)
    table = br.find_by_xpath(tr_selector)
    try:
        if len(table) > 0:
            temp_dict = {}
            temp_dict['player'] = []
            temp_dict['pos'] = []
            temp_dict['salary'] = []
            temp_dict['start'] = []
            temp_dict['mins'] = []
            temp_dict['score'] = []
            for ind, row in enumerate(table):
                temp_dict['player'].append(row.find_by_css(".player-stats").text)
                temp_dict['salary'].append(row.find_by_css(".salaryCol").text)
                try:
                    temp_dict['pos'].append(row.find_by_css("td select.pos-select option")[0].text)
                except:
                    temp_dict['pos'].append(row.find_by_css("td")[1].text)
                temp_dict['start'].append(row.find_by_css("td")[-4].text)
                temp_dict['mins'].append(row.find_by_css("td")[-3].text)
                temp_dict['score'].append(row.find_by_css("td")[-2].text)                              
            temp_df = pd.DataFrame(temp_dict)
            temp_df['date']=date
            salary_data2 = salary_data2.append(temp_df)
        else:
            print("empty {}".format(date))
    except:
        print("error {}".format(date))
print("Done!")

2017-10-17
error 2017-10-18


ProtocolError: ('Connection aborted.', BrokenPipeError(32, 'Broken pipe'))

In [144]:
data = pd.read_csv('2017Data.csv',encoding='latin-1')
temp_df = pd.read_csv(years[1],encoding='latin-1')
data.append(temp_df)
temp_df = pd.read_csv(years[2],encoding='latin-1')
data.append(temp_df)
data.tail()

,Unnamed: 0,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,...,name,offensive_rebounds,opponent,outcome,personal_fouls,seconds_played,slug,steals,team,turnovers
28591,167,0.0,5.0,0.0,0.0,0.0,2017-12-31,0.0,-1.5,Location.AWAY,...,Shabazz Muhammad,0.0,Team.INDIANA_PACERS,Outcome.WIN,1.0,227.0,muhamsh01,0.0,Team.MINNESOTA_TIMBERWOLVES,0.0
28592,168,2.0,9.0,2.0,1.0,0.0,2017-12-31,9.0,-1.7,Location.AWAY,...,Dwight Howard,1.0,Team.LOS_ANGELES_CLIPPERS,Outcome.LOSS,4.0,2049.0,howardw01,2.0,Team.CHARLOTTE_HORNETS,5.0
28593,169,1.0,4.0,2.0,0.0,0.0,2017-12-31,3.0,-2.4,Location.HOME,...,Kosta Koufos,2.0,Team.MEMPHIS_GRIZZLIES,Outcome.LOSS,2.0,598.0,koufoko01,1.0,Team.SACRAMENTO_KINGS,2.0
28594,170,0.0,6.0,0.0,3.0,0.0,2017-12-31,2.0,-3.4,Location.AWAY,...,Marvin Williams,0.0,Team.LOS_ANGELES_CLIPPERS,Outcome.LOSS,2.0,992.0,willima02,2.0,Team.CHARLOTTE_HORNETS,1.0
28595,171,1.0,5.0,1.0,1.0,2.0,2017-12-31,2.0,-4.4,Location.HOME,...,Myles Turner,0.0,Team.MINNESOTA_TIMBERWOLVES,Outcome.LOSS,4.0,1148.0,turnemy01,0.0,Team.INDIANA_PACERS,3.0


In [145]:
years = ['2017Data.csv','2018Data.csv','2019Data.csv']

for year in years:
    temp_df = pd.read_csv(year,encoding='latin-1')
    data = data.append(temp_df)
data = data.iloc[:,1:]
data.tail()

,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,made_field_goals,...,name,offensive_rebounds,opponent,outcome,personal_fouls,seconds_played,slug,steals,team,turnovers
16171,0.0,1.0,0.0,1.0,0.0,2019-06-13,1.0,0.2,Location.HOME,0.0,...,Alfonzo McKinnie,2.0,Team.TORONTO_RAPTORS,Outcome.LOSS,2.0,629.0,mckinal01,0.0,Team.GOLDEN_STATE_WARRIORS,0.0
16172,0.0,0.0,0.0,0.0,0.0,2019-06-13,0.0,0.0,Location.HOME,0.0,...,Jonas Jerebko,0.0,Team.TORONTO_RAPTORS,Outcome.LOSS,0.0,139.0,jerebjo01,0.0,Team.GOLDEN_STATE_WARRIORS,0.0
16173,0.0,3.0,0.0,2.0,0.0,2019-06-13,1.0,-0.2,Location.HOME,1.0,...,Quinn Cook,0.0,Team.TORONTO_RAPTORS,Outcome.LOSS,2.0,750.0,cookqu01,0.0,Team.GOLDEN_STATE_WARRIORS,0.0
16174,0.0,1.0,0.0,0.0,0.0,2019-06-13,1.0,-0.4,Location.HOME,0.0,...,Andrew Bogut,0.0,Team.TORONTO_RAPTORS,Outcome.LOSS,0.0,170.0,bogutan01,0.0,Team.GOLDEN_STATE_WARRIORS,0.0
16175,0.0,2.0,0.0,1.0,0.0,2019-06-13,1.0,-2.9,Location.AWAY,0.0,...,Norman Powell,0.0,Team.GOLDEN_STATE_WARRIORS,Outcome.WIN,2.0,650.0,powelno01,0.0,Team.TORONTO_RAPTORS,1.0


In [194]:
#dates = data[data['date']>'2017-09-01']['date'].unique()
dates = data[data['date']>'2018-03-07']['date'].unique()
dates[0]

'2018-03-08'

In [ ]:
salary_data = pd.DataFrame()
for date in dates[0:1]:
    time.sleep(3)
    br.visit(rewind_base+date)

In [161]:
salary_data = pd.DataFrame()
for date in dates[0:1]:
    time.sleep(3)
    br.visit(rewind_base+date)
    

2
0


KeyboardInterrupt: 

In [289]:
import src.fantasy_scoring as fs

In [290]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 101280 entries, 0 to 16175
Data columns (total 22 columns):
assists                              101280 non-null float64
attempted_field_goals                101280 non-null float64
attempted_free_throws                101280 non-null float64
attempted_three_point_field_goals    101280 non-null float64
blocks                               101280 non-null float64
date                                 101280 non-null object
defensive_rebounds                   101280 non-null float64
game_score                           101280 non-null float64
location                             101280 non-null object
made_field_goals                     101280 non-null float64
made_free_throws                     101280 non-null float64
made_three_point_field_goals         101280 non-null float64
name                                 101280 non-null object
offensive_rebounds                   101280 non-null float64
opponent                             10

In [299]:
%load_ext autoreload
%autoreload 2

In [341]:
from src.cleaner import Cleaner 
import src.fantasy_scoring as fs

In [342]:
clnr = Cleaner()
data = clnr.bb_ref_to_dk(data)
data['dk_score']= data.apply(lambda x: fs.dk_score(x),axis = 1)

In [343]:
len(data[data['dd']==1])/len(data)

0.08135860979462875

In [349]:
data[(data['date']>='2019-05-01')&(data['name']=='Kawhi Leonard')]

,assists,attempted_field_goals,attempted_free_throws,attempted_three_point_field_goals,blocks,date,defensive_rebounds,game_score,location,made_field_goals,...,slug,steals,team,turnovers,points,rebounds,doubles,dd,td,dk_score
15495,3.0,22.0,5.0,4.0,1.0,2019-05-02,1.0,22.2,Location.AWAY,13.0,...,leonaka01,0.0,Team.TORONTO_RAPTORS,5.0,33.0,4.0,1,0,0,43.00
15575,5.0,20.0,12.0,7.0,0.0,2019-05-05,11.0,29.7,Location.AWAY,13.0,...,leonaka01,0.0,Team.TORONTO_RAPTORS,7.0,39.0,14.0,2,1,0,64.50
15652,4.0,16.0,8.0,4.0,0.0,2019-05-07,11.0,21.3,Location.HOME,7.0,...,leonaka01,2.0,Team.TORONTO_RAPTORS,0.0,21.0,13.0,2,1,0,48.75
15747,5.0,20.0,12.0,4.0,0.0,2019-05-09,11.0,21.9,Location.AWAY,9.0,...,leonaka01,1.0,Team.TORONTO_RAPTORS,4.0,29.0,12.0,2,1,0,53.00
15807,3.0,39.0,8.0,9.0,0.0,2019-05-12,4.0,24.4,Location.HOME,16.0,...,leonaka01,3.0,Team.TORONTO_RAPTORS,4.0,41.0,8.0,1,0,0,60.50
15868,2.0,26.0,10.0,5.0,0.0,2019-05-15,8.0,20.5,Location.AWAY,10.0,...,leonaka01,3.0,Team.TORONTO_RAPTORS,3.0,31.0,9.0,1,0,0,50.25
15905,2.0,18.0,10.0,4.0,1.0,2019-05-17,7.0,23.7,Location.AWAY,10.0,...,leonaka01,0.0,Team.TORONTO_RAPTORS,2.0,31.0,8.0,1,0,0,45.50
15952,5.0,25.0,13.0,4.0,1.0,2019-05-19,7.0,26.4,Location.HOME,11.0,...,leonaka01,2.0,Team.TORONTO_RAPTORS,5.0,36.0,9.0,1,0,0,59.25
15995,1.0,13.0,8.0,3.0,2.0,2019-05-21,5.0,17.7,Location.HOME,6.0,...,leonaka01,4.0,Team.TORONTO_RAPTORS,2.0,19.0,7.0,1,0,0,40.75
16017,9.0,25.0,9.0,8.0,0.0,2019-05-23,4.0,30.5,Location.AWAY,11.0,...,leonaka01,2.0,Team.TORONTO_RAPTORS,1.0,35.0,7.0,1,0,0,63.25


In [347]:
data[data['date']=='2019-05-16'][['name','dk_score','date','points','made_three_point_field_goals','made_field_goals','made_free_throws',
         'rebounds','assists','steals','blocks','turnovers','dd','td']]

,name,dk_score,date,points,made_three_point_field_goals,made_field_goals,made_free_throws,rebounds,assists,steals,blocks,turnovers,dd,td
15882,Stephen Curry,58.00,2019-05-16,37.0,4.0,11.0,11.0,8.0,8.0,0.0,0.0,6.0,0,0
15883,Damian Lillard,49.25,2019-05-16,23.0,5.0,6.0,6.0,5.0,10.0,1.0,0.0,2.0,1,0
15884,Kevon Looney,29.75,2019-05-16,14.0,0.0,6.0,2.0,7.0,2.0,1.0,1.0,0.0,0,0
15885,Seth Curry,29.75,2019-05-16,16.0,4.0,5.0,2.0,1.0,2.0,4.0,0.0,1.0,0,0
15886,Draymond Green,48.00,2019-05-16,16.0,0.0,8.0,0.0,10.0,7.0,0.0,5.0,5.0,1,0
15887,Maurice Harkless,23.00,2019-05-16,12.0,2.0,4.0,2.0,4.0,2.0,1.0,0.0,0.0,0,0
15888,Klay Thompson,31.75,2019-05-16,24.0,4.0,8.0,4.0,3.0,2.0,0.0,0.0,2.0,0,0
15889,CJ McCollum,34.75,2019-05-16,22.0,3.0,9.0,1.0,3.0,5.0,1.0,0.0,4.0,0,0
15890,Rodney Hood,17.00,2019-05-16,12.0,2.0,4.0,2.0,2.0,1.0,0.0,0.0,0.0,0,0
15891,Jordan Bell,21.25,2019-05-16,11.0,0.0,4.0,3.0,3.0,1.0,2.0,1.0,2.0,0,0


In [370]:
#salary_data2 = pd.read_csv('data/salary_data2018.csv')
salary_data = pd.read_csv('data/salary_data2017.csv')
#salary_data = salary_data.append(salary_data2)


#
salary_data.tail()

,player,pos,salary,date
17926,Devin Robinson,SG,3000,2017-12-31
17927,TJ Leaf,SF/PF,3000,2017-12-31
17928,PJ Dozier,PG/SG,3000,2017-12-31
17929,Daniel Hamilton,SF,3000,2017-12-31
17930,Jacob Wiley,PG,3000,2017-12-31


In [367]:
salary_data.to_csv('data/salary_data2017.csv',index = False)
salary_data.to_csv('data/salary_data2017backup.csv',index = False)

In [356]:
salary_data.to_csv('data/salary_data2019backup.csv',index = False)